In [11]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import nltk
from bs4 import BeautifulSoup
import re,string

import os
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
emails=pd.read_csv("/content/gdrive/MyDrive/emails.csv")

In [14]:
emails.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [15]:
#Collating all functions together and applying them for the 'IMDb reviews' dataset
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing emojis
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols                                                                         
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

#Text-encoding: UTF-8 encoder
def to_unicode(text):
    if isinstance(text, float):
        text = str(text)
    if isinstance(text, int):
        text = str(text)
    if not isinstance(text, str):
        text = text.decode('utf-8', 'ignore')
    return text

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)


#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Removing the noisy text
def denoise_text(text):
    text = to_unicode(text)
    text = strip_html(text)
    text = re.sub(r"http\S+", "", text)
    text = deEmojify(text)
    text = text.encode('ascii', 'ignore')
    text = to_unicode(text)
    text = remove_between_square_brackets(text)
    text = remove_special_characters(text)
    text = text.lower()
    return text


In [16]:
emails['text']=emails['text'].apply(denoise_text)

In [17]:
#Removing stopwords
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer

#Tokenization of text
tokenizer=ToktokTokenizer() #for every function

#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [18]:
from nltk.tokenize import word_tokenize,sent_tokenize


stop=set(stopwords.words('english'))
print(stop)

#Removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

#Apply function on review column
emails['text']=emails['text'].apply(remove_stopwords)

{'m', "couldn't", 'him', 'he', 'will', 'same', 'very', 'wouldn', "hadn't", 'they', 'aren', 'about', 'just', 'its', 's', 'by', 'were', "aren't", 'needn', 'am', 'where', 'again', 'don', 'hasn', "that'll", 'once', 'which', 'if', "isn't", 'ma', 'ours', 'himself', 'their', 'when', 'y', 'so', "weren't", 'she', 'what', 'themselves', 'most', "haven't", "didn't", 'until', 'now', 'how', 'for', 'whom', 'against', 'at', 'that', 'all', 'but', 'on', "don't", 'we', 'shouldn', 'those', "mustn't", 'doing', 'with', 'further', 'such', 'own', 'won', 'ourselves', "you'd", 'you', 'into', 'hadn', 'can', 'who', 'his', 'should', 'there', 'had', 'up', 'isn', 'why', 'during', 'out', 'did', 'over', 'other', 'my', 'll', 'not', 'is', 'itself', "you've", "won't", 'haven', 'doesn', 'as', 'any', 'o', 'too', 'here', 'after', 'this', "should've", 're', 'i', "you'll", 'does', 'didn', "hasn't", 'herself', 'below', 'the', "needn't", 'd', 'has', 'while', 'them', 'before', 'hers', 'wasn', "mightn't", 'then', 'or', 'through',

In [19]:
#Stemming and Lemmatization
from nltk.stem import WordNetLemmatizer,SnowballStemmer
nltk.download('wordnet')

def simple_stemmer(text):
    ps=SnowballStemmer(language='english')
    return ' '.join([ps.stem(word) for word in tokenizer.tokenize(text)])

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [20]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

#Lemmatizer example
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        else:
            yield word
            
def lemmatize_text(text):
    return ' '.join(lemmatize_all(text))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [21]:
emails['text']=emails['text'].apply(lemmatize_text)

LookupError: ignored

In [22]:
#Creating features using Bag of words model and building the Logistic Regression model
#Transformed train reviews
norm_text=emails.text

In [23]:
from sklearn.preprocessing import LabelBinarizer

#Labelling the sentient data
lb=LabelBinarizer()

#Transformed sentiment data
spam=lb.fit_transform(emails['spam'])
print(spam.shape)

(5728, 1)


In [24]:
#Binarisation of spam or ham: ham: 0, spam: 1

In [25]:
#Fitting the bag of words model on the entire dataset
from sklearn.feature_extraction.text import CountVectorizer

#Creating a matrix with reviews in row and unique words as columns and frequency of word in review as values.
#Count vectorizer for bag of words
cv=CountVectorizer()

#Fitting model on entire data
cv_fit = cv.fit(norm_text)

In [26]:
#Normalised train reviews
norm_train_text=emails.text[:3500]
print('train:','\n',norm_train_text[0])
norm_train_cv_text=cv_fit.transform(norm_train_text)

#Normalised test reviews
norm_test_text=emails.text[3500:]
print('test:','\n',norm_test_text[3501])
norm_test_cv_text=cv_fit.transform(norm_test_text)

train: 
 subject naturally irresistible corporate identity lt really hard recollect company market full suqgestions information isoverwhelminq good catchy logo stylish statlonery outstanding website make task much easier promise havinq ordered iogo company automaticaily become world ieader isguite ciear without good products effective business organization practicable aim hotat nowadays market promise marketing efforts become much effective list clear benefits creativeness hand made original logos specially done reflect distinctive company image convenience logo stationery provided formats easy use content management system letsyou change website content even structure promptness see logo drafts within three business days affordability marketing break make gaps budget 100 satisfaction guaranteed provide unlimited amount changes extra fees surethat love result collaboration look portfolio _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

In [27]:
spam_train=spam[:3500]
spam_test=spam[3500:]

In [28]:
emails.spam.value_counts()

0    4360
1    1368
Name: spam, dtype: int64

In [29]:
from keras.models import Sequential
from keras.layers import Dense

In [30]:
n_t=norm_train_cv_text.shape[1]

In [31]:
model=Sequential()
model.add(Dense(50,input_shape=(n_t,),activation='relu'))
model.add(Dense(20,activation='relu'))
#model.add(Dense(25,activation='relu'))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
history=model.fit(norm_train_cv_text,spam_train,epochs=10,verbose=2,validation_data=(norm_test_cv_text,spam_test))

Epoch 1/10
110/110 - 6s - loss: 0.1792 - accuracy: 0.9469 - val_loss: 0.0261 - val_accuracy: 0.9969 - 6s/epoch - 50ms/step
Epoch 2/10
110/110 - 1s - loss: 0.0160 - accuracy: 0.9994 - val_loss: 0.0209 - val_accuracy: 0.9969 - 628ms/epoch - 6ms/step
Epoch 3/10
110/110 - 1s - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.0264 - val_accuracy: 0.9955 - 619ms/epoch - 6ms/step
Epoch 4/10
110/110 - 1s - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0267 - val_accuracy: 0.9951 - 607ms/epoch - 6ms/step
Epoch 5/10
110/110 - 1s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0321 - val_accuracy: 0.9937 - 615ms/epoch - 6ms/step
Epoch 6/10
110/110 - 1s - loss: 7.8581e-04 - accuracy: 1.0000 - val_loss: 0.0424 - val_accuracy: 0.9919 - 608ms/epoch - 6ms/step
Epoch 7/10
110/110 - 1s - loss: 4.9205e-04 - accuracy: 1.0000 - val_loss: 0.0483 - val_accuracy: 0.9915 - 613ms/epoch - 6ms/step
Epoch 8/10
110/110 - 1s - loss: 3.3630e-04 - accuracy: 1.0000 - val_loss: 0.0582 - val_accuracy: 0.9910 - 628ms/epoch -

In [33]:
loss,acc=model.evaluate(norm_test_cv_text,spam_test)
print("acccuracy",acc*100,"%")

70/70 [==============================] - 0s 3ms/step - loss: 0.0727 - accuracy: 0.9910
acccuracy 99.10233616828918 %


In [34]:
#unseeen external data
t={'text':["AASIF MOHAMMAD,Thank you for participating "]}
y=pd.DataFrame.from_dict(t,orient='columns')
test=cv_fit.transform(y['text'])

In [35]:
model.predict(test[0])

array([[0.1435123]], dtype=float32)

In [36]:
from sklearn.linear_model import LogisticRegression,SGDClassifier

#Training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)

#Fitting the model for the bag of words
lr_bow=lr.fit(norm_train_cv_text,spam_train)
print(lr_bow)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [37]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(norm_test_cv_text)
print(lr_bow_predict)

[0 0 0 ... 0 0 0]


####unseen external data**

In [38]:
#unseeen external data
d={'text':["you will recive the bitcoins worth $3000 plz share  your bank details with us"]}
x=pd.DataFrame.from_dict(d,orient='columns')
test_1=cv_fit.transform(x['text'])

In [39]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(test_1)
print(lr_bow_predict)

[1]


In [40]:
frames=[y,x]
data=pd.concat(frames,axis=0,ignore_index=True)
#data=data.reshape(1,)

test_2=cv_fit.transform(data['text'])
#data[0][0]
data.head()

,text
0,"AASIF MOHAMMAD,Thank you for participating"
1,you will recive the bitcoins worth $3000 plz s...


In [41]:
test_2.toarray().shape

  #print(i[x])
data_1=pd.DataFrame({"data":[data['text'][0],data['text'][1]]})
  #x=x+1
#data['text'][0]

In [42]:
test_2.toarray().shape
data_1.head()
#data['text'][1]

,data
0,"AASIF MOHAMMAD,Thank you for participating"
1,you will recive the bitcoins worth $3000 plz s...


In [43]:
#Predicting the model for bag of words
#for i in data[0]:
  #da=cv_fit.fit_transform(i)
lr_bow_predict=lr.predict(test_2)
print(lr_bow_predict)
lr_bow_predict[0]
lr_bow_predict[1]

[0 1]


1

In [44]:

d={"pred":[lr_bow_predict[0],lr_bow_predict[1]]}
#data_1.head()

In [45]:
#lr_bow_predict.reshape(1,)
prediction=pd.DataFrame.from_dict(d,orient="columns",)
prediction.head()

,pred
0,0
1,1


In [46]:
#fram=[data_1,prediction]
pred_data=pd.concat([data_1,prediction],join='outer',axis=1)
pred_data.head()

,data,pred
0,"AASIF MOHAMMAD,Thank you for participating",0
1,you will recive the bitcoins worth $3000 plz s...,1


In [49]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

#Accuracy score for bag of words
lr_bow_score=accuracy_score(spam_test,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)

ValueError: ignored